In [1]:
%%capture

!pip install --upgrade mne-bids-pipeline
!pip install numpy matplotlib scipy numba scikit-learn mne PyWavelets pandas
!pip install mne-features

In [2]:
%%capture

from mne_features.feature_extraction import extract_features
import mne
from joblib import Parallel, delayed
import pandas as pd

# Initialize an empty DataFrame to store the features
feature_df = pd.DataFrame()

# Load the participant data
participants = pd.read_csv('../participants.tsv', sep='\t')

# List of all subject ids
#subject_ids = participants['participant_id'].unique()

subject_ids = ['sub-101', 'sub-102', 'sub-103', 'sub-104',
              'sub-105', 'sub-106', 'sub-107', 'sub-108',
              'sub-109', 'sub-110', 'sub-111']

# Frequency bands defined in the paper
frequency_bands = [0, 2, 4, 8, 13, 18, 24, 30, 49]

# List of functions to be applied from the paper
selected_funcs = [
    'mean', 'std', 'kurtosis', 'skewness', 'quantile', 'ptp_amp',
    'pow_freq_bands', 'spect_entropy', 'app_entropy', 'samp_entropy', 
    'hurst_exp', 'hjorth_complexity', 'hjorth_mobility', 'line_length',
    'wavelet_coef_energy', 'higuchi_fd', 'zero_crossings', 'svd_fisher_info'
]

# Parameters for the feature functions
selected_params = {
    'quantile__q': [0.1, 0.25, 0.75, 0.9],
    'pow_freq_bands__freq_bands': frequency_bands,
}

def compute_features(i, ch_name, epochs):
    # Extract the data for this channel
    ch_data = epochs.get_data(picks=ch_name)
    
    # Compute the features
    features = extract_features(ch_data, sfreq=epochs.info['sfreq'], selected_funcs=selected_funcs, funcs_params=selected_params, return_as_df=False)
    
    # Initialize a dictionary to store the features
    feature_dict = {}

    # Unpack features and add them to the feature_dict
    for feature_name, feature_values in zip(selected_funcs, features):
        feature_dict[f'{ch_name}_{feature_name}'] = feature_values[0]
    
    return feature_dict

# Loop over the subjects
for sub_id in subject_ids:
    # Load the data for current subject
    epochs = mne.read_epochs(f'../derivatives/{sub_id}/ses-t1/eeg/{sub_id}_ses-t1_task-resteyesc_proc-clean_epo.fif')
    
    # Compute the features for all channels in parallel
    results = Parallel(n_jobs=-1, backend='loky')(delayed(compute_features)(i, ch_name, epochs) for i, ch_name in enumerate(epochs.ch_names))
    
    # Initialize a dictionary to store the features
    feature_dict = {}

    # Concatenate results
    for result in results:
        feature_dict.update(result)

    # Get the age of the current subject
    subject_age = participants.loc[participants['participant_id'] == sub_id, 'age'].values[0]

    # Add the age to the dictionary
    feature_dict['age'] = subject_age

    # Convert dictionary to DataFrame and append it to feature_df
    feature_df = feature_df.append(pd.DataFrame(feature_dict, index=[0]), ignore_index=True)


/usr/local/lib/python3.9/dist-packages/mne_features/utils.py:21: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit()
/usr/local/lib/python3.9/dist-packages/mne_features/utils.py:21: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit()
/usr/local/lib/python3.9/dist-packages/mne_features/utils.py:21: NumbaDeprecationWarning: The 'nopython' keyword argument 

In [3]:
feature_df.to_csv('feature_df_sub-101-111.csv', index=False)

In [1]:
df = pd.read_csv('feature_df_sub-071-080.csv')

In [2]:
df.head() # 1153 columns ?

,Fp1_mean,Fp1_std,Fp1_kurtosis,Fp1_skewness,Fp1_quantile,Fp1_ptp_amp,Fp1_pow_freq_bands,Fp1_spect_entropy,Fp1_app_entropy,Fp1_samp_entropy,...,O2_samp_entropy,O2_hurst_exp,O2_hjorth_complexity,O2_hjorth_mobility,O2_line_length,O2_wavelet_coef_energy,O2_higuchi_fd,O2_zero_crossings,O2_svd_fisher_info,age
0,-0.005874,-0.005911,-0.005949,-0.006011,-0.006026,-0.006027,-0.006007,-0.005985,-0.006026,-0.006039,...,0.000904,0.000892,0.000900,0.000933,0.000974,0.001026,0.001049,0.001070,0.001101,43
1,-0.006812,-0.006845,-0.006838,-0.006860,-0.006863,-0.006864,-0.006867,-0.006850,-0.006830,-0.006881,...,-0.002159,-0.002130,-0.002120,-0.002113,-0.002107,-0.002117,-0.002139,-0.002146,-0.002137,40
2,-0.002490,-0.002440,-0.002437,-0.002434,-0.002456,-0.002460,-0.002495,-0.002484,-0.002503,-0.002523,...,0.000875,0.000886,0.000944,0.000985,0.000996,0.000957,0.000952,0.000890,0.000849,21
3,0.004733,0.004705,0.004684,0.004687,0.004674,0.004643,0.004623,0.004621,0.004621,0.004619,...,0.000126,0.000162,0.000177,0.000207,0.000211,0.000234,0.000232,0.000232,0.000280,32
4,-0.001961,-0.001894,-0.001936,-0.001878,-0.001926,-0.001973,-0.001827,-0.001849,-0.001862,-0.001880,...,0.006793,0.006512,0.006315,0.006184,0.006085,0.006010,0.005940,0.005898,0.005825,24


In [3]:
df.shape

(10, 1153)

In [1]:
# importing pandas
import pandas as pd

# merging two csv files
df = pd.concat(
    map(pd.read_csv, ['feature_df_sub-001.csv', 'feature_df_sub-002-008.csv',
                     'feature_df_sub-009-015.csv', 'feature_df_sub-016-026.csv',
                     'feature_df_sub-027-029.csv', 'feature_df_sub-030-035.csv',
                     'feature_df_sub-036-040.csv', 'feature_df_sub-041-050.csv',
                     'feature_df_sub-051-054.csv', 'feature_df_sub-055-070.csv',
                     'feature_df_sub-071-080.csv', 'feature_df_sub-081-090.csv',
                     'feature_df_sub-091-100.csv', 'feature_df_sub-101-111.csv']), ignore_index=True)

df.to_csv('df.csv', index=False)

In [4]:
import pandas as pd

df = pd.read_csv('df.csv')
df.head()

,Fp1_mean,Fp1_std,Fp1_kurtosis,Fp1_skewness,Fp1_quantile,Fp1_ptp_amp,Fp1_pow_freq_bands,Fp1_spect_entropy,Fp1_app_entropy,Fp1_samp_entropy,...,O2_samp_entropy,O2_hurst_exp,O2_hjorth_complexity,O2_hjorth_mobility,O2_line_length,O2_wavelet_coef_energy,O2_higuchi_fd,O2_zero_crossings,O2_svd_fisher_info,age
0,-0.009561,-0.009513,-0.009498,-0.009499,-0.009477,-0.009442,-0.009491,-0.009563,-0.009492,-0.009530,...,-0.001292,-0.001309,-0.001307,-0.001299,-0.001319,-0.001342,-0.001337,-0.001353,-0.001350,29
1,0.000566,0.000523,0.000540,0.000566,0.000589,0.000574,0.000573,0.000570,0.000576,0.000553,...,0.001427,0.001429,0.001420,0.001410,0.001401,0.001432,0.001439,0.001433,0.001404,29
2,-0.000193,-0.000195,-0.000187,-0.000181,-0.000162,-0.000210,-0.000180,-0.000174,-0.000206,-0.000216,...,0.000237,0.000250,0.000270,0.000314,0.000335,0.000314,0.000365,0.000367,0.000342,62
3,-0.009172,-0.009178,-0.009152,-0.009125,-0.009099,-0.009066,-0.009074,-0.009058,-0.009043,-0.009040,...,0.002945,0.002904,0.002897,0.002907,0.002900,0.002891,0.002911,0.002874,0.002835,20
4,-0.001474,-0.001508,-0.001375,-0.001295,-0.001307,-0.001228,-0.001088,-0.001179,-0.001078,-0.001097,...,0.002836,0.002819,0.002829,0.002838,0.002815,0.002836,0.002815,0.002779,0.002789,32
